In [1]:
# lab-model-generation-and-validation


In [5]:
import pandas as pd
import numpy as np
from scipy import stats

In [6]:
data = pd.read_csv("marketing_customer_analysis.csv")

In [25]:
# Identify columns with string values
string_cols = data.select_dtypes(include=['object']).columns

# Drop columns with string values from the dataset
data_numeric = data.drop(string_cols, axis=1)

# Apply z-score calculation on the numeric dataset
data_numeric = data_numeric[(np.abs(stats.zscore(data_numeric)) < 3).all(axis=1)]

In [26]:
# Print the column names
print(data.columns)



Index(['Customer', 'State', 'Customer Lifetime Value', 'Response', 'Coverage',
       'Education', 'Effective To Date', 'EmploymentStatus', 'Gender',
       'Income', 'Location Code', 'Marital Status', 'Monthly Premium Auto',
       'Months Since Last Claim', 'Months Since Policy Inception',
       'Number of Open Complaints', 'Number of Policies', 'Policy Type',
       'Policy', 'Renew Offer Type', 'Sales Channel', 'Total Claim Amount',
       'Vehicle Class', 'Vehicle Size'],
      dtype='object')


In [27]:
X = data.drop("Total Claim Amount", axis=1)  
y = data["Total Claim Amount"]

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [29]:
X_train_df = pd.DataFrame(X_train, columns=X.columns)
X_test_df = pd.DataFrame(X_test, columns=X.columns)


In [30]:
numerical_cols = X_train.select_dtypes(include=[np.number]).columns
categorical_cols = X_train.select_dtypes(include=["object"]).columns

X_train_numerical = X_train_df[numerical_cols]
X_train_categorical = X_train_df[categorical_cols]


In [38]:
from sklearn.preprocessing import MinMaxScaler
import pickle
import os

scaler = MinMaxScaler()
X_train_numerical_scaled = scaler.fit_transform(X_train_numerical)

path = "transformers/"
isExist = os.path.exists(path)

if not isExist:
    os.makedirs(path)
    print("The new directory is created!")

filename = "filename.pkl"  # Use a descriptive name for your scaler/transformer but keep the ".pkl" file extension
with open(path + filename, "wb") as file:
    pickle.dump(scaler, file)

The new directory is created!


In [13]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown="ignore")
X_train_categorical_encoded = encoder.fit_transform(X_train_categorical)

path = "/"
os.makedirs(path, exist_ok=True)
filename = "categorical_encoder.pkl"
with open(path + filename, "wb") as file:
    pickle.dump(encoder, file)


NameError: name 'X_train_categorical' is not defined

In [40]:
X_train_transformed = pd.concat([pd.DataFrame(X_train_numerical_scaled, columns=numerical_cols),
                                pd.DataFrame(X_train_categorical_encoded.toarray(), columns=encoder.get_feature_names_out())],
                                axis=1)


In [42]:
final_scaler = MinMaxScaler()
X_train_final_scaled = final_scaler.fit_transform(X_train_transformed)


path = "scalers/"
os.makedirs(path, exist_ok=True)
filename = "final_scaler.pkl"
with open(os.path.join(path, filename), "wb") as file:
    pickle.dump(final_scaler, file)

In [10]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def error_metrics_report(y_real_train: list, y_real_test: list, y_pred_train: list, y_pred_test: list) -> pd.DataFrame:
    '''
        Calculates various error metrics for regression models and returns a Pandas DataFrame.

    y_real_train (list): The actual values for the training set.
    y_real_test (list): The actual values for the test set.
    y_pred_train (list): The predicted values for the training set.
    y_pred_test (list): The predicted values for the test set.

Returns:
    pd.DataFrame: DataFrame containing the error metrics for the training and test sets.
                  The columns are "Metric", "Train", and "Test".

    '''

    MAE_train = mean_absolute_error(y_real_train, y_pred_train)
    MAE_test = mean_absolute_error(y_real_test, y_pred_test)

    # Mean squared error
    MSE_train = mean_squared_error(y_real_train, y_pred_train)
    MSE_test = mean_squared_error(y_real_test, y_pred_test)

    # Root mean squared error
    RMSE_train = mean_squared_error(y_real_train, y_pred_train, squared=False)
    RMSE_test = mean_squared_error(y_real_test, y_pred_test, squared=False)


    # R2
    R2_train = r2_score(y_real_train, y_pred_train)
    R2_test = r2_score(y_real_test, y_pred_test)
    
    
    results = {"Metric": ["MAE", "MSE", "RMSE", "R2"],
    "Train": [MAE_train, MSE_train, RMSE_train, R2_train],
    "Test": [MAE_test, MSE_test, RMSE_test, R2_test]}

    results_df = pd.DataFrame(results).round(2)

    return results_df

In [11]:
#error_metrics_report(list(results_df[results_df["Set"]=="Train"]["Real"]),
                     #list(results_df[results_df["Set"]=="Test"]["Real"]),
                     #list(results_df[results_df["Set"]=="Train"]["Predicted"]),
                     #list(results_df[results_df["Set"]=="Test"]["Predicted"]))

In [12]:
#error_metrics_report(list('results_df'[results_df["Set"]=="Train"]["Real"]),
                     #list('results_df'[results_df["Set"]=="Test"]["Real"]),
                     #list('results_df'[results_df["Set"]=="Train"]["Predicted"]),
                     #list('results_df'[results_df["Set"]=="Test"]["Predicted"]))

#print.matrix_df()

In [9]:
results_df = pd.DataFrame(columns=["Set", "Real", "Predicted"])

# Populate the DataFrame with values
results_df["Set"] = ["Train"] * len(y_real_train) + ["Test"] * len(y_real_test)
results_df["Real"] = y_real_train + y_real_test
results_df["Predicted"] = y_pred_train + y_pred_test

# Call the error_metrics_report function
metrics_df = error_metrics_report(list(results_df[results_df["Set"] == "Train"]["Real"]),
                                  list(results_df[results_df["Set"] == "Test"]["Real"]),
                                  list(results_df[results_df["Set"] == "Train"]["Predicted"]),
                                  list(results_df[results_df["Set"] == "Test"]["Predicted"]))

# Print the result
print(metrics_df)

NameError: name 'y_real_train' is not defined